In [16]:
import glob
import xarray as xr 

import os 

In [18]:
files = glob.glob('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/daily/ta/pdSST-pdSIC/AWI-CM-1-1-MR/*')

names = os.path.basename(files[0])
names 

'ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r100i1p1f1_gn_20000601-20010531.nc'

In [15]:
test = xr.open_mfdataset(files[0])
test

<xarray.Dataset>
Dimensions:    (time: 365, bnds: 2, plev: 19, lat: 192, lon: 384)
Coordinates:
  * time       (time) datetime64[ns] 2000-06-01T12:00:00 ... 2001-05-31T12:00:00
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * lat        (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * lon        (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(384, 2), meta=np.ndarray>
    ta         (time, plev, lat, lon) float32 dask.array<chunksize=(1, 10, 96, 192), meta=np.ndarray>
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            PAMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    creation_date:          2020-07-31T02:51:38Z
    ...                     ...
    variable_id:            ta
    variant_label:          r100i1p1f1
    license:                CMIP6 model data produced by Alfred Wegener Insti...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/adbd5d62-7855-4218-8840-623ed1002bcb
    variant_info:           AWI/MPI atmosphere-only PAMIP simulations are not...

In [19]:
for count, item in enumerate(files):
    
    names = os.path.basename(files[count])
    print(names)
    
    # print(files[count])
    
    # ds = xr.open_mfdataset(item)
    # print(len(ds.lat.values))

ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r100i1p1f1_gn_20000601-20010531.nc
ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r10i1p1f1_gn_20000601-20010531.nc
ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r11i1p1f1_gn_20000601-20010531.nc
ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r12i1p1f1_gn_20000601-20010531.nc
ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r13i1p1f1_gn_20000601-20010531.nc
ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r14i1p1f1_gn_20000601-20010531.nc
ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r15i1p1f1_gn_20000601-20010531.nc
ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r16i1p1f1_gn_20000601-20010531.nc
ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r17i1p1f1_gn_20000601-20010531.nc
ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r18i1p1f1_gn_20000601-20010531.nc
ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r19i1p1f1_gn_20000601-20010531.nc
ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r1i1p1f1_gn_20000601-20010531.nc
ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r20i1p1f1_gn_20000601-20010531.nc
ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r21i1p1f1_gn_20000601-20010531.nc
ta_Eday_AWI-CM-1-1-MR_pdSST-pdSIC_r22i1p1f1_gn_2

In [12]:
def regrid_dataset_3x3(ds, check_dims=False):
    
    """
    Input: Xarray dataset
            - Must contain (lat, lon)
            
    Output: Regridded Xarray dataset 
            to 3 deg lat lon
    
    """
    
    import sys
    sys.path.append('/home/users/cturrell/documents/eddy_feedback/')
    import functions.data_wrangling as data 
    
    import xesmf as xe 
    import numpy as np 
    
    # Rename 
    if check_dims:
        ds = data.check_dimensions(ds)
        
    
    # build regridder
    ds_out = xr.Dataset(
        {
            'lat': (['lat'], np.arange(-90, 93, 3)),
            'lon': (['lon'], np.arange(0,360, 3))
        }
    )

    regridder = xe.Regridder(ds, ds_out, "bilinear")
    ds_new = regridder(ds)

    print('Dataset regridded. Checking variables...')

    # verify that the result is the same as regridding each variable one-by-one
    for k in ds.data_vars:
        print(k, ds_new[k].equals(regridder(ds[k])))

    print('Checks complete. Dataset ready.')
    
    return ds_new 

In [14]:
test = test[['ta']]

ds = regrid_dataset_3x3(test, check_dims=True)
ds 

Dataset regridded. Checking variables...
ta True
Checks complete. Dataset ready.


<xarray.Dataset>
Dimensions:  (time: 365, level: 19, lat: 61, lon: 120)
Coordinates:
  * time     (time) datetime64[ns] 2000-06-01T12:00:00 ... 2001-05-31T12:00:00
  * level    (level) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * lat      (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon      (lon) int64 0 3 6 9 12 15 18 21 ... 336 339 342 345 348 351 354 357
Data variables:
    ta       (time, level, lat, lon) float32 dask.array<chunksize=(1, 10, 61, 120), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [20]:
new_files = glob.glob('/gws/nopw/j04/arctic_connect/cturrell/PAMIP_data/regridded/AWI-CM-1-1-MR_3x3/*')

ds = xr.open_mfdataset(new_files[0])
ds 

<xarray.Dataset>
Dimensions:  (time: 365, level: 19, lat: 61, lon: 120)
Coordinates:
  * time     (time) datetime64[ns] 2000-06-01T12:00:00 ... 2001-05-31T12:00:00
  * level    (level) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * lat      (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon      (lon) int64 0 3 6 9 12 15 18 21 ... 336 339 342 345 348 351 354 357
Data variables:
    ta       (time, level, lat, lon) float32 dask.array<chunksize=(365, 19, 61, 120), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear